In [4]:
import pandas as pd
import numpy as np
import operator
import itertools
from itertools import combinations, chain
def PreprocessCTC(data):
    df=data
    for idx, row in df.iterrows():
        if row.CTC > 100:
            data.set_value(idx, 'CTC', row.CTC/100000)
    return(data)

def TierProcess(data):
    df_ctc=PreprocessCTC(data)
    df_ctc['CTC'] = pd.to_numeric(df_ctc['CTC'],errors='coerce')
    #map CTC to Tier level based on 2021 Tier mapping
    for idx, row in data.iterrows():
        if np.isnan(row.TierLevel):
            #print(row.CTC) #mostly all 0 are printed here
            if row.CTC > 0 and row.CTC <= 5:
                df_ctc.set_value(idx, 'TierLevel', 3)
            if row.CTC > 5 and row.CTC <= 10:
                df_ctc.set_value(idx, 'TierLevel', 2)
            if row.CTC > 10:
                df_ctc.set_value(idx, 'TierLevel', 1)
    return(df_ctc)

In [5]:
master=pd.read_csv('UpdatedMaster19Feb_NLP.csv')
master1 = master.drop_duplicates(subset=['USN','CompanyName_New'], keep='last')

In [6]:
df_final=TierProcess(master1)
df_final.CompanyName_New.unique()

<ipython-input-4-d8b6d95616ae>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ctc['CTC'] = pd.to_numeric(df_ctc['CTC'],errors='coerce')


array(['Walmart Labs', 'TheMathCompany', 'NXP Semiconductors',
       'Novarties Healthcare', 'Optum', 'Intuit', 'Cisco Systems',
       'Raramuri Technology', 'Rupeek Fintech', 'Enquero Global',
       'AirAsia Technology Centre', 'Logmein', 'Samsung',
       'Akamai Technologies', 'IBM', 'Mathworks', nan, 'Accenture',
       'Wipro', 'LAM Research', 'Goldman Sachs', 'National Instruments',
       'App Dynamics', 'Societe Generale', 'Morgan Stanley',
       'Pluribus Networks', 'Sandvine', 'Microsoft', 'PWC', 'Commvault',
       'Cure.Fit', 'Epsilon', 'Hewlett-Packard Enterprise',
       'Zscaler Softech', 'Robert Bosch Engineering & Business Solutions',
       'Amadeus Software Labs', 'Couchbase', 'Cloudera', 'Deloitte',
       'Amazon', 'SymphonyAI', 'Kickdrum Technologies', 'Amagi', 'Groww',
       'L&T Technology Services', 'Siemens Healthneers', 'Citrix',
       'Flipkart', 'MakeMyTrip', 'Pensando Systems', 'NCR Corporations',
       'Gameskraft Technologies', 'SAP Labs', 'Target

In [7]:
tier1 = df_final.loc[df_final['TierLevel'] == 1]

In [8]:
tier1 = df_final.loc[df_final['TierLevel'] == 1]

In [9]:
tier1.CompanyName_New.unique()

array(['Walmart Labs', 'NXP Semiconductors', 'Novarties Healthcare',
       'Optum', 'Intuit', 'Cisco Systems', 'Raramuri Technology',
       'Rupeek Fintech', 'AirAsia Technology Centre', 'Samsung',
       'Akamai Technologies', 'Mathworks', nan, 'Goldman Sachs',
       'App Dynamics', 'Societe Generale', 'Morgan Stanley',
       'Pluribus Networks', 'Sandvine', 'Microsoft', 'Commvault',
       'Cure.Fit', 'Epsilon', 'Hewlett-Packard Enterprise',
       'Zscaler Softech', 'Amadeus Software Labs', 'Couchbase',
       'Cloudera', 'Amazon', 'SymphonyAI', 'Kickdrum Technologies',
       'Amagi', 'Groww', 'Citrix', 'Flipkart', 'MakeMyTrip',
       'Pensando Systems', 'NCR Corporations', 'Gameskraft Technologies',
       'SAP Labs', 'Target', 'Analog Devices', 'PWC', 'LAM Research',
       'IBM', 'Deloitte', 'Adobe', 'Logmein'], dtype=object)

In [10]:
companydomainsdf=pd.read_csv('Company Domains - Sheet1.csv')
df_merged = pd.merge(tier1, companydomainsdf, on=["CompanyName_New"])


In [34]:
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
def find_SetsCompanyDomain(Domain_Name,number):
    data= df_merged[['ProjectDetailDomain',Domain_Name]]
    data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')
    data = data.dropna()
    if(number==1):
        data=data.loc[data[Domain_Name] == 1]
    if(number!=1):
        Domain_name=['Algorithms','Data Structures','ProjectDetailDomain']
        data= df_merged[Domain_name]
        data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')
        data = data.dropna()
        data=data.loc[(data['Data Structures'] == 1) & (data['Algorithms']== 1)]
    domain_count={}
    for v,k in data['ProjectDetailDomain'].iteritems():
        for i in k:
            if i in domain_count:
                domain_count[i]=domain_count[i]+1
            else:
                domain_count[i]=1
    domain_count
    try:
        domain_count.pop('')
    except:
        placeholder=1
    domain_sup={}
    for k, v in domain_count.items():
        domain_sup[k] = v/272
    a=dict(sorted(domain_count.items(), key=lambda item: item[1]))
    #calulate MIS for each itemset

    #Step 1: find SD
    #Lambda(mean)
    lamb = 0
    for val in domain_count.values(): 
        lamb += val 
    lamb = lamb / len(domain_count) 
    #Consider alpha(between 1 and 0) 0.5
    alpha= 0.5
    sd=lamb*(1-alpha)
    #S and MIS
    ls=8
    mis_val=mis(domain_count,ls,alpha,lamb) #dictionary with item and MIS
    #mis_val 
    #domain_count    # dictionary with support count and item
    mis_min=min(mis_val.values())
    L={}
    for k, v in domain_count.items():
        if(domain_count[k]>mis_val[k]):
            L[k]=mis_val[k]
    #Step 2
    #Sort L
    temp= sorted(L.items(), key=operator.itemgetter(0))
    L1=sorted(temp, key=operator.itemgetter(1), reverse=True)
    n = 2
    C2=findsubsets(L1, n)
    count={}
    for i,v in C2:
        for index, row in data.iterrows():
            t=""
            if(i[0] in row['ProjectDetailDomain'] and v[0] in row['ProjectDetailDomain']):
                t=i[0]+","+v[0]
                if(t in count):
                    count[t]=count[t]+1
                else:
                    count[t]=1

                #checking criteria for count
    L2={}
    for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L2[k]=count[k]
            
    #FOR TIER 1, SORTED 2 ITEMSETS 
    temp= sorted(L2.items(), key=operator.itemgetter(0))
    L2_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
    print(L2_sorted)
    n = 3
    C3=findsubsets(L1, n)
    count={}
    for a,b,c in C3:
        for index, row in data.iterrows():
            t=""
            if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain']):
                t=a[0]+","+b[0]+','+c[0]
                if(t in count):
                    count[t]=count[t]+1
                else:
                    count[t]=1
                
    L3={}
    for k, v in count.items():
            m=count[k]*100/272
            if(m>mis_min):
                L3[k]=count[k]
            
    #FOR TIER 1, SORTED 3 ITEMSETS 
    temp= sorted(L3.items(), key=operator.itemgetter(0))
    L3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
    print(L3_sorted)
    n = 4
    C4=findsubsets(L1, n)
    count={}
    for a,b,c,d in C4:
        for index, row in data.iterrows():
            t=""
            if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain'] and d[0] in row['ProjectDetailDomain']):
                t=a[0]+","+b[0]+','+c[0]+','+d[0]
                if(t in count):
                    count[t]=count[t]+1
                else:
                    count[t]=1

    L4={}
    for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L4[k]=count[k]
            
    #FOR TIER 1, SORTED 4 ITEMSETS 
    temp= sorted(L4.items(), key=operator.itemgetter(0))
    L4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
    print(L4_sorted)

In [35]:
def mis(domain_count,ls,alpha,lamb):
    sd=lamb*(1-alpha)
    mis=[]
    misdict={}
    mis_val={}
    for k, v in domain_count.items():
        m=domain_count[k]-sd
        if(m<ls):
            mis.append(ls)
            misdict[domain_count[k]]=ls
            mis_val[k]=ls
        else:
            mis.append(m)
            misdict[domain_count[k]]=m
            mis_val[k]=m
    #print(mis_val)
    return mis_val


In [36]:
find_SetsCompanyDomain('Algorithms',1)


<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[('Cloud computing,Image Processing', 41), ('Cloud computing,Database Management System', 33), ('Database Management System,Image Processing', 32), ('Cloud computing,Web Development', 31), ('Cloud computing,Algorithms', 26), ('Cloud computing,Data Structures', 25), ('Cloud computing,Machine Learning', 25), ('Cloud computing,Compiler Design', 24), ('Database Management System,Web Development', 24), ('Cloud computing,Operating System', 23), ('Cloud computing,Deep Learning', 22), ('Web Development,Image Processing', 22)]
[('Cloud computing,Database Management System,Image Processing', 25)]
[]


In [37]:
find_SetsCompanyDomain('Data Structures',1)

<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[('Image Processing,Cloud computing', 53), ('Database Management System,Cloud computing', 43), ('Web Development,Cloud computing', 41), ('Image Processing,Database Management System', 38), ('Algorithms,Cloud computing', 36), ('Machine Learning,Cloud computing', 33), ('Data Structures,Cloud computing', 32), ('Compiler Design,Cloud computing', 31), ('Operating System,Cloud computing', 30), ('Deep Learning,Cloud computing', 29), ('Database Management System,Web Development', 28), ('Image Processing,Web Development', 28), ('Image Processing,Algorithms', 27), ('Machine Learning,Image Processing', 27), ('Database Management System,Algorithms', 26), ('Database Management System,Data Structures', 26), ('Image Processing,Data Structures', 24), ('Machine Learning,Algorithms', 24), ('Compiler Design,Database Management System', 23), ('Operating System,Database Management System', 23), ('Operating System,Image Processing', 22)]
[('Image Processing,Database Management System,Cloud computing', 31), 

In [38]:
find_SetsCompanyDomain('Database Management Systems',1)

[('Image Processing,Cloud computing', 24)]


<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[]
[]


In [39]:
find_SetsCompanyDomain('Operating Systems',1)

<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[('Cloud computing,Image Processing', 39), ('Cloud computing,Database Management System', 31), ('Web Development,Cloud computing', 30), ('Cloud computing,Algorithms', 28), ('Database Management System,Image Processing', 26), ('Cloud computing,Operating System', 25), ('Cloud computing,Machine Learning', 24), ('Cloud computing,Data Structures', 22), ('Deep Learning,Cloud computing', 22)]
[]
[]


In [40]:
find_SetsCompanyDomain('Web Development',1)

[('Cloud computing,Image Processing', 29)]


<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[]
[]


In [41]:
find_SetsCompanyDomain('Data Structures',2)

[('Image Processing,Cloud computing', 40), ('Database Management System,Cloud computing', 33), ('Image Processing,Database Management System', 32), ('Cloud computing,Web Development', 31), ('Cloud computing,Algorithms', 26), ('Cloud computing,Machine Learning', 25), ('Data Structures,Cloud computing', 25), ('Cloud computing,Compiler Design', 24), ('Database Management System,Web Development', 24), ('Deep Learning,Cloud computing', 22), ('Image Processing,Web Development', 22), ('Operating System,Cloud computing', 22)]
[('Database Management System,Image Processing,Cloud computing', 25)]


<ipython-input-34-94a62b762717>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')
<ipython-input-34-94a62b762717>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ProjectDetailDomain']=data['ProjectDetailDomain'].str.split('##')


[]


In [109]:
#2 ITEMSETS 


#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row['ProjectDetailDomain'] and v[0] in row['ProjectDetailDomain']):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L2={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L2[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L2.items(), key=operator.itemgetter(0))
L2_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L2_sorted

[('Cloud computing,Image Processing', 41),
 ('Database Management System,Cloud computing', 33),
 ('Database Management System,Image Processing', 32),
 ('Web Development,Cloud computing', 31),
 ('Algorithms,Cloud computing', 26),
 ('Cloud computing,Data Structures', 25),
 ('Machine Learning,Cloud computing', 25),
 ('Cloud computing,Compiler Design', 24),
 ('Database Management System,Web Development', 24),
 ('Operating System,Cloud computing', 23),
 ('Cloud computing,Deep Learning', 22),
 ('Web Development,Image Processing', 22)]

In [110]:
#3 ITEMSETS

n = 3
C3=findsubsets(L1, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain']):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L3.items(), key=operator.itemgetter(0))
L3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L3_sorted


[('Database Management System,Cloud computing,Image Processing', 25)]

In [111]:
#4 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 4
C4=findsubsets(L1, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain'] and d[0] in row['ProjectDetailDomain']):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L4.items(), key=operator.itemgetter(0))
L4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L4_sorted

[]

In [112]:
#5 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 5
C5=findsubsets(L1, n)

count={}
for a,b,c,d,e in C5:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain'] and d[0] in row['ProjectDetailDomain'] and e[0] in row['ProjectDetailDomain']):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]+','+e[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L5={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min):
            L5[k]=count[k]
            
#FOR TIER 1, SORTED 5 ITEMSETS 
temp= sorted(L5.items(), key=operator.itemgetter(0))
L5_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L5_sorted

[]

In [113]:
#changing alpha and LS values 
#Lambda is mean value

alpha2= 0.9
#sd=lamb*(1-alpha2)
ls=15
mis_val2=mis(domain_count,ls,alpha)
mis_min2=min(mis_val2.values())

L_2={}
for k, v in domain_count.items():
        if(domain_count[k]>=mis_val2[k]):
            L_2[k]=mis_val2[k]

#Step 2
#Sort L
temp= sorted(L_2.items(), key=operator.itemgetter(0))
L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)

#2 ITEMSETS 
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
n = 2
C2=findsubsets(L_21, n)

#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row['ProjectDetailDomain'] and v[0] in row['ProjectDetailDomain']):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L_12={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_12[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L_12.items(), key=operator.itemgetter(0))
L_12_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_12_sorted

[('Cloud computing,Image Processing', 41)]

In [114]:
#3 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 3
C3=findsubsets(L_21, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain']):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L_3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L_3.items(), key=operator.itemgetter(0))
L_3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_3_sorted

[]

In [115]:
#4 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 4
C4=findsubsets(L_21, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProjectDetailDomain'] and b[0] in row['ProjectDetailDomain'] and c[0] in row['ProjectDetailDomain'] and d[0] in row['ProjectDetailDomain']):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L_4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L_4.items(), key=operator.itemgetter(0))
L_4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_4_sorted

[]